In [1]:
from langchain.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question.
{schema}

{question}
QL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [2]:
from langchain.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///../data/Chinook.db")

def get_schema(_):
    return db.get_table_info()

def run_query(query):
    return db.run(query)


In [3]:
import boto3
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.llms import Bedrock

bedrock_runtime = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')
model = Bedrock(
            model_id="cohere.command-text-v14",
            client=bedrock_runtime)

sql_response = (
        RunnablePassthrough.assign(schema=get_schema)
        | prompt
        | model.bind(stop=["\nSQLResult:"])
        | StrOutputParser()
    )

In [4]:
result = sql_response.invoke({"question": "How many employees are there?"})

/Users/thandavm/anaconda3/lib/python3.11/site-packages/langchain/utilities/sql_database.py:357: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  sample_rows_result = connection.execute(command)  # type: ignore


In [5]:
print(result)

 SELECT count(*) FROM Employee


In [6]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

In [7]:
full_chain = (
    RunnablePassthrough.assign(query=sql_response) 
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response 
    | model
)

In [10]:
full_chain.invoke({"question": "List any 3 albums and their artist?"})

OperationalError: (sqlite3.OperationalError) no such column: A.AlbumId
[SQL:  SELECT  A.AlbumId, A.Title, A.ArtistId, B.Name
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)